In [0]:

from zipfile import ZipFile
zip_test = ZipFile("test1.zip")
zip_test.extractall()
zip_test.close()


In [0]:
zip_train = ZipFile("train.zip")
zip_train.extractall()
zip_train.close()

In [0]:
import os
from tqdm import tqdm 

'''Labelling the dataset'''
def label_img(img): 
    word_label = img.split('.')[-3] 
    # DIY One hot encoder 
    if word_label == 'cat': return 0 
    elif word_label == 'dog': return 1 

In [0]:
import cv2 
import numpy as np
from random import shuffle
  

In [0]:

  
'''Creating the training data'''
def create_train_data(): 
    # Creating an empty list where we should the store the training data 
    # after a little preprocessing of the data 
    training_data = [] 
  
    # tqdm is only used for interactive loading 
    # loading the training data 
    for img in tqdm(os.listdir("train")): 
  
        # labeling the images 
        label = label_img(img) 
  
        path = os.path.join("train", img) 
#         print(path)
        # loading the image from the path and then converting them into 
        # greyscale for easier covnet prob 
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE) 
  
        # resizing the image for processing them in the covnet 
        img = cv2.resize(img, (64, 64)) 
  
        # final step-forming the training data list with numpy array of the images 
        training_data.append([np.array(img), np.array(label)]) 
  
    # shuffling of the training data to preserve the random state of our data 
    shuffle(training_data) 
  
    # saving our trained data for further uses if required 
    np.save('train_data.npy', training_data) 
    return training_data 
  

'''Running the training and the testing in the dataset for our model'''
train_data = create_train_data() 



FileNotFoundError: ignored

In [0]:
import numpy as np
data = np.load('train_data.npy')


In [4]:
train = data[:-500]
test = data[-500:]
train.shape

(24500, 2)

In [0]:
X_train = np.array([i[0] for i in train]).reshape(-1,64,64,1)
X_test= np.array([i[0] for i in test]).reshape(-1,64,64,1)
y_train = np.array([i[1] for i in train])
y_test = np.array([i[1] for i in test])

In [6]:
y_train.shape

(24500,)

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers import Dense,Flatten

# Model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3),  activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


model.add(Conv2D(32, (3, 3),  activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(layers.Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train, epochs=70, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model.predict_classes(X_train) 
y_predicted_test_catDog = model.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/70
24500/24500 [==============================] - 11s 467us/step - loss: 0.7995 - acc: 0.5260 - val_loss: 0.6861 - val_acc: 0.5300
Epoch 2/70
24500/24500 [==============================] - 11s 446us/step - loss: 0.6864 - acc: 0.5525 - val_loss: 0.6575 - val_acc: 0.6360
Epoch 3/70
24500/24500 [==============================] - 10s 426us/step - loss: 0.6236 - acc: 0.6552 - val_loss: 0.5521 - val_acc: 0.7440
Epoch 4/70
24500/24500 [==============================] - 10s 424us/step - loss: 0.5565 - acc: 0.7219 - val_loss: 0.5379 - val_acc: 0.7280
Epoch 5/70
24500/24500 [==============================] - 10s 424us/step - loss: 0.5193 - acc: 0.7466 - val_loss: 0.4592 - val_acc: 0.7700
Epoch 6/70
24500/24500 [==============================] - 10s 425us/step - loss: 0.4796 - acc: 0.7758 - val_loss: 0.4165 - val_acc: 0.8020
Epoch 7/70
24500/24500 [==============================] - 10s 422us/step - loss: 0.4562 - acc: 0.7885 - val_loss: 0.39

In [0]:
model.summary()
model.save('initialized-model.keras')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_65 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_66 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_67 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 4, 4, 64)          18496     
__________

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers import Dense,Flatten

# Model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3),  activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


model.add(Conv2D(32, (3, 3),  activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Flatten())
model.add(layers.Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train, epochs=50, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model.predict_classes(X_train) 
y_predicted_test_catDog = model.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/50
24500/24500 [==============================] - 12s 490us/step - loss: 0.7521 - acc: 0.5596 - val_loss: 0.6889 - val_acc: 0.5140
Epoch 2/50
24500/24500 [==============================] - 11s 450us/step - loss: 0.6562 - acc: 0.6087 - val_loss: 0.6297 - val_acc: 0.6920
Epoch 3/50
24500/24500 [==============================] - 11s 448us/step - loss: 0.6077 - acc: 0.6715 - val_loss: 0.5379 - val_acc: 0.7500
Epoch 4/50
24500/24500 [==============================] - 11s 449us/step - loss: 0.5494 - acc: 0.7239 - val_loss: 0.5067 - val_acc: 0.7680
Epoch 5/50
24500/24500 [==============================] - 11s 447us/step - loss: 0.5052 - acc: 0.7550 - val_loss: 0.4759 - val_acc: 0.7640
Epoch 6/50
24500/24500 [==============================] - 11s 449us/step - loss: 0.4673 - acc: 0.7814 - val_loss: 0.4745 - val_acc: 0.7860
Epoch 7/50
24500/24500 [==============================] - 11s 450us/step - loss: 0.4262 - acc: 0.8039 - val_loss: 0.44

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers import Dense,Flatten

# Model
model2 = Sequential()

model2.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model2.add(MaxPooling2D(pool_size = (2, 2)))

model2.add(Conv2D(32, (3, 3),  activation = 'relu'))
model2.add(MaxPooling2D(pool_size = (2, 2)))


model2.add(Conv2D(32, (3, 3),  activation = 'relu'))
model2.add(MaxPooling2D(pool_size = (2, 2)))


model2.add(Conv2D(64, (3, 3)))
model2.add(MaxPooling2D(pool_size = (2, 2)))
model2.add(Flatten())
model2.add(layers.Dropout(0.8))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model2.fit(X_train,y_train, epochs=80, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model2.predict_classes(X_train) 
y_predicted_test_catDog = model2.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/80
24500/24500 [==============================] - 13s 514us/step - loss: 1.0954 - acc: 0.5170 - val_loss: 0.6912 - val_acc: 0.5100
Epoch 2/80
24500/24500 [==============================] - 11s 461us/step - loss: 0.6908 - acc: 0.5149 - val_loss: 0.6933 - val_acc: 0.5000
Epoch 3/80
24500/24500 [==============================] - 11s 469us/step - loss: 0.6905 - acc: 0.5142 - val_loss: 0.6907 - val_acc: 0.5040
Epoch 4/80
24500/24500 [==============================] - 11s 459us/step - loss: 0.6878 - acc: 0.5227 - val_loss: 0.6906 - val_acc: 0.4900
Epoch 5/80
24500/24500 [==============================] - 11s 458us/step - loss: 0.6896 - acc: 0.5161 - val_loss: 0.6882 - val_acc: 0.4980
Epoch 6/80
24500/24500 [==============================] - 11s 458us/step - loss: 0.6877 - acc: 0.5230 - val_loss: 0.6953 - val_acc: 0.4800
Epoch 7/80
24500/24500 [==============================] - 11s 460us/step - loss: 0.6870 - acc: 0.5287 - val_loss: 0.68

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers import Dense,Flatten

# Model
model3 = Sequential()

model3.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model3.add(MaxPooling2D(pool_size = (2, 2)))

model3.add(Conv2D(32, (3, 3),  activation = 'relu'))
model3.add(MaxPooling2D(pool_size = (2, 2)))


model3.add(Conv2D(32, (3, 3),  activation = 'relu'))
model3.add(MaxPooling2D(pool_size = (2, 2)))


model3.add(Conv2D(64, (3, 3)))
model3.add(MaxPooling2D(pool_size = (2, 2)))
model3.add(Flatten())
model3.add(layers.Dropout(0.2))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model3.fit(X_train,y_train, epochs=130, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model3.predict_classes(X_train) 
y_predicted_test_catDog = model3.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/130
24500/24500 [==============================] - 13s 520us/step - loss: 0.7577 - acc: 0.5240 - val_loss: 0.6849 - val_acc: 0.5380
Epoch 2/130
24500/24500 [==============================] - 11s 451us/step - loss: 0.6839 - acc: 0.5460 - val_loss: 0.6918 - val_acc: 0.5620
Epoch 3/130
24500/24500 [==============================] - 11s 450us/step - loss: 0.6845 - acc: 0.5392 - val_loss: 0.6974 - val_acc: 0.5300
Epoch 4/130
24500/24500 [==============================] - 11s 453us/step - loss: 0.6846 - acc: 0.5450 - val_loss: 0.6933 - val_acc: 0.4900
Epoch 5/130
24500/24500 [==============================] - 11s 452us/step - loss: 0.6821 - acc: 0.5517 - val_loss: 0.6721 - val_acc: 0.5720
Epoch 6/130
24500/24500 [==============================] - 11s 454us/step - loss: 0.6751 - acc: 0.5728 - val_loss: 0.6490 - val_acc: 0.6380
Epoch 7/130
24500/24500 [==============================] - 11s 456us/step - loss: 0.6279 - acc: 0.6488 - val_los

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers import Dense,Flatten

# Model
model4 = Sequential()

model4.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model4.add(MaxPooling2D(pool_size = (2, 2)))

model4.add(Conv2D(32, (3, 3),  activation = 'relu'))
model4.add(MaxPooling2D(pool_size = (2, 2)))


model4.add(Conv2D(32, (3, 3),  activation = 'relu'))
model4.add(MaxPooling2D(pool_size = (2, 2)))


model4.add(Conv2D(64, (3, 3)))
model4.add(MaxPooling2D(pool_size = (2, 2)))
model4.add(Flatten())
model4.add(Dense(64, activation='relu'))
model4.add(Dense(64, activation='relu'))
model4.add(Dense(32, activation='relu'))
model4.add(layers.Dropout(0.5))
model4.add(Dense(1, activation='sigmoid'))


model4.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model4.fit(X_train,y_train, epochs=100, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model4.predict_classes(X_train) 
y_predicted_test_catDog = model4.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/100
24500/24500 [==============================] - 13s 535us/step - loss: 0.6955 - acc: 0.5550 - val_loss: 0.6488 - val_acc: 0.6340
Epoch 2/100
24500/24500 [==============================] - 12s 493us/step - loss: 0.6165 - acc: 0.6718 - val_loss: 0.5142 - val_acc: 0.7360
Epoch 3/100
24500/24500 [==============================] - 12s 486us/step - loss: 0.5400 - acc: 0.7373 - val_loss: 0.5039 - val_acc: 0.7420
Epoch 4/100
24500/24500 [==============================] - 12s 493us/step - loss: 0.4875 - acc: 0.7738 - val_loss: 0.4354 - val_acc: 0.7820
Epoch 5/100
24500/24500 [==============================] - 12s 488us/step - loss: 0.4452 - acc: 0.7996 - val_loss: 0.3875 - val_acc: 0.8140
Epoch 6/100
24500/24500 [==============================] - 12s 489us/step - loss: 0.4104 - acc: 0.8193 - val_loss: 0.3375 - val_acc: 0.8440
Epoch 7/100
24500/24500 [==============================] - 12s 489us/step - loss: 0.3786 - acc: 0.8366 - val_los

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers import Dense,Flatten

# Model
model5 = Sequential()

model5.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model5.add(MaxPooling2D(pool_size = (2, 2)))

model5.add(Conv2D(32, (3, 3),  activation = 'relu'))
model5.add(MaxPooling2D(pool_size = (2, 2)))


model5.add(Conv2D(32, (3, 3),  activation = 'relu'))
model5.add(MaxPooling2D(pool_size = (2, 2)))


model5.add(Conv2D(64, (3, 3)))
model5.add(MaxPooling2D(pool_size = (2, 2)))
model5.add(Flatten())
model5.add(Dense(64, activation='relu'))
model5.add(Dense(64, activation='relu'))
model5.add(Dense(64, activation='relu'))
model5.add(layers.Dropout(0.7))
model5.add(Dense(64, activation='relu'))
model5.add(Dense(32, activation='relu'))
model5.add(layers.Dropout(0.7))
model5.add(Dense(1, activation='sigmoid'))


model5.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model5.fit(X_train,y_train, epochs=100, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model5.predict_classes(X_train) 
y_predicted_test_catDog = model5.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/100
24500/24500 [==============================] - 15s 630us/step - loss: 0.7193 - acc: 0.4977 - val_loss: 0.6928 - val_acc: 0.5280
Epoch 2/100
24500/24500 [==============================] - 14s 565us/step - loss: 0.6931 - acc: 0.5089 - val_loss: 0.6922 - val_acc: 0.5860
Epoch 3/100
24500/24500 [==============================] - 14s 566us/step - loss: 0.6916 - acc: 0.5265 - val_loss: 0.6917 - val_acc: 0.5280
Epoch 4/100
24500/24500 [==============================] - 14s 566us/step - loss: 0.6883 - acc: 0.5421 - val_loss: 0.6820 - val_acc: 0.5600
Epoch 5/100
24500/24500 [==============================] - 14s 561us/step - loss: 0.6736 - acc: 0.5848 - val_loss: 0.6416 - val_acc: 0.6580
Epoch 6/100
24500/24500 [==============================] - 14s 560us/step - loss: 0.6387 - acc: 0.6510 - val_loss: 0.6255 - val_acc: 0.6640
Epoch 7/100
24500/24500 [==============================] - 14s 560us/step - loss: 0.6110 - acc: 0.6827 - val_los

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model6 = Sequential()

model6.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model6.add(MaxPooling2D(pool_size = (2, 2)))

model6.add(Conv2D(32, (3, 3),  activation = 'relu'))
model6.add(MaxPooling2D(pool_size = (2, 2)))


model6.add(Conv2D(32, (3, 3),  activation = 'relu'))
model6.add(MaxPooling2D(pool_size = (2, 2)))


model6.add(Conv2D(64, (3, 3)))
model6.add(MaxPooling2D(pool_size = (2, 2)))
model6.add(BatchNormalization())
model6.add(Dropout(0.2))

model6.add(Flatten())
model6.add(Dense(128, activation='relu'))
model6.add(Dense(64, activation='relu'))
model6.add(Dense(64, activation='relu'))
model6.add(layers.Dropout(0.7))
model6.add(Dense(64, activation='relu'))
model6.add(Dense(32, activation='relu'))
model6.add(layers.Dropout(0.7))
model6.add(Dense(1, activation='sigmoid'))


model6.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model6.fit(X_train,y_train, epochs=100, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model6.predict_classes(X_train) 
y_predicted_test_catDog = model6.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/100
24500/24500 [==============================] - 17s 682us/step - loss: 0.7030 - acc: 0.5191 - val_loss: 0.6841 - val_acc: 0.5660
Epoch 2/100
24500/24500 [==============================] - 15s 595us/step - loss: 0.6798 - acc: 0.5722 - val_loss: 0.6623 - val_acc: 0.6580
Epoch 3/100
24500/24500 [==============================] - 15s 605us/step - loss: 0.6350 - acc: 0.6413 - val_loss: 0.5835 - val_acc: 0.6560
Epoch 4/100
24500/24500 [==============================] - 15s 600us/step - loss: 0.5495 - acc: 0.7310 - val_loss: 0.9355 - val_acc: 0.5700
Epoch 5/100
24500/24500 [==============================] - 15s 601us/step - loss: 0.4771 - acc: 0.7884 - val_loss: 0.4122 - val_acc: 0.8220
Epoch 6/100
24500/24500 [==============================] - 15s 597us/step - loss: 0.4366 - acc: 0.8146 - val_loss: 0.4389 - val_acc: 0.7840
Epoch 7/100
24500/24500 [==============================] - 15s 597us/step - loss: 0.4015 - acc: 0.8312 - val_los

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model7 = Sequential()

model7.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model7.add(MaxPooling2D(pool_size = (2, 2)))

model7.add(Conv2D(64, (3, 3),  activation = 'relu'))
model7.add(MaxPooling2D(pool_size = (2, 2)))


model7.add(Conv2D(128, (3, 3),  activation = 'relu'))
model7.add(MaxPooling2D(pool_size = (2, 2)))


model7.add(Conv2D(64, (3, 3)))
model7.add(MaxPooling2D(pool_size = (2, 2)))
model7.add(BatchNormalization())


model7.add(Flatten())
model7.add(Dense(128, activation='relu'))
model7.add(BatchNormalization())
model7.add(Dense(64, activation='relu'))
model7.add(layers.Dropout(0.7))
model7.add(Dense(128, activation='relu'))
model7.add(Dense(64, activation='relu'))
model7.add(Dense(128, activation='relu'))
model7.add(BatchNormalization())
model7.add(Dense(64, activation='relu'))
model7.add(layers.Dropout(0.7))
model7.add(Dense(1, activation='sigmoid'))


model7.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model7.fit(X_train,y_train, epochs=150, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model7.predict_classes(X_train) 
y_predicted_test_catDog = model7.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/150
24500/24500 [==============================] - 23s 958us/step - loss: 0.7410 - acc: 0.5317 - val_loss: 0.6850 - val_acc: 0.5900
Epoch 2/150
24500/24500 [==============================] - 22s 915us/step - loss: 0.6538 - acc: 0.6287 - val_loss: 0.7289 - val_acc: 0.5600
Epoch 3/150
24500/24500 [==============================] - 23s 919us/step - loss: 0.5606 - acc: 0.7301 - val_loss: 1.1664 - val_acc: 0.5000
Epoch 4/150
24500/24500 [==============================] - 23s 925us/step - loss: 0.4904 - acc: 0.7820 - val_loss: 0.5841 - val_acc: 0.5940
Epoch 5/150
24500/24500 [==============================] - 23s 923us/step - loss: 0.4435 - acc: 0.8116 - val_loss: 0.5352 - val_acc: 0.7200
Epoch 6/150
24500/24500 [==============================] - 23s 928us/step - loss: 0.4148 - acc: 0.8240 - val_loss: 0.6882 - val_acc: 0.6700
Epoch 7/150
24500/24500 [==============================] - 23s 925us/step - loss: 0.3828 - acc: 0.8417 - val_los

In [0]:
model6.summary()
model6.save('initialized-model.keras')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_45 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 12, 12, 32)        9248      
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 4, 4, 64)          18496     
__________

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model8 = Sequential()

model8.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model8.add(MaxPooling2D(pool_size = (2, 2)))

model8.add(Conv2D(64, (3, 3),  activation = 'relu'))
model8.add(MaxPooling2D(pool_size = (2, 2)))


model8.add(Conv2D(128, (3, 3),  activation = 'relu'))
model8.add(MaxPooling2D(pool_size = (2, 2)))



model8.add(Flatten())
model8.add(Dense(128, activation='relu'))
model8.add(BatchNormalization())
model8.add(layers.Dropout(0.5))
model8.add(Dense(1, activation='sigmoid'))


model8.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model8.fit(X_train,y_train, epochs=50, batch_size=32, validation_data = (X_test, y_test) )



y_predicted_train_catDog = model8.predict_classes(X_train) 
y_predicted_test_catDog = model8.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/50
24500/24500 [==============================] - 19s 784us/step - loss: 0.5892 - acc: 0.7016 - val_loss: 0.4937 - val_acc: 0.7700
Epoch 2/50
24500/24500 [==============================] - 14s 585us/step - loss: 0.4465 - acc: 0.7938 - val_loss: 0.6942 - val_acc: 0.6940
Epoch 3/50
24500/24500 [==============================] - 14s 584us/step - loss: 0.3807 - acc: 0.8328 - val_loss: 0.4145 - val_acc: 0.8060
Epoch 4/50
24500/24500 [==============================] - 14s 582us/step - loss: 0.3237 - acc: 0.8591 - val_loss: 0.2984 - val_acc: 0.8900
Epoch 5/50
24500/24500 [==============================] - 14s 583us/step - loss: 0.2864 - acc: 0.8792 - val_loss: 0.3655 - val_acc: 0.8380
Epoch 6/50
24500/24500 [==============================] - 14s 585us/step - loss: 0.2455 - acc: 0.8983 - val_loss: 0.3173 - val_acc: 0.8740
Epoch 7/50
24500/24500 [==============================] - 14s 583us/step - loss: 0.2099 - acc: 0.9138 - val_loss: 0.35

In [10]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model9 = Sequential()

model9.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model9.add(MaxPooling2D(pool_size = (2, 2)))

model9.add(Conv2D(32, (3, 3),  activation = 'relu'))
model9.add(MaxPooling2D(pool_size = (2, 2)))


model9.add(Conv2D(64, (3, 3),  activation = 'relu'))
model9.add(MaxPooling2D(pool_size = (2, 2)))


model9.add(Flatten())
model9.add(layers.Dropout(0.5))
model9.add(Dense(1, activation='sigmoid'))


model9.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model9.fit(X_train,y_train, epochs=50, batch_size=32, validation_data = (X_test, y_test) )

y_predicted_train_catDog = model9.predict_classes(X_train) 
y_predicted_test_catDog = model9.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/50
24500/24500 [==============================] - 10s 392us/step - loss: 0.8068 - acc: 0.5329 - val_loss: 0.6822 - val_acc: 0.5500
Epoch 2/50
24500/24500 [==============================] - 9s 375us/step - loss: 0.6945 - acc: 0.5469 - val_loss: 0.6893 - val_acc: 0.5440
Epoch 3/50
24500/24500 [==============================] - 9s 375us/step - loss: 0.6866 - acc: 0.5610 - val_loss: 0.6724 - val_acc: 0.5860
Epoch 4/50
24500/24500 [==============================] - 9s 375us/step - loss: 0.6753 - acc: 0.5987 - val_loss: 0.6458 - val_acc: 0.6140
Epoch 5/50
24500/24500 [==============================] - 9s 376us/step - loss: 0.6331 - acc: 0.6511 - val_loss: 0.5889 - val_acc: 0.7000
Epoch 6/50
24500/24500 [==============================] - 9s 372us/step - loss: 0.5932 - acc: 0.6903 - val_loss: 0.5829 - val_acc: 0.7080
Epoch 7/50
24500/24500 [==============================] - 9s 372us/step - loss: 0.5642 - acc: 0.7168 - val_loss: 0.4952 - v

In [15]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model10 = Sequential()

model10.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model10.add(MaxPooling2D(pool_size = (2, 2)))

model10.add(Conv2D(64, (3, 3),  activation = 'relu'))
model10.add(MaxPooling2D(pool_size = (2, 2)))


model10.add(Conv2D(64, (3, 3),  activation = 'relu'))
model10.add(MaxPooling2D(pool_size = (2, 2)))


model10.add(Flatten())
model10.add(Dense(1, activation='sigmoid'))


model10.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model10.fit(X_train,y_train, epochs=50, batch_size=32, validation_data = (X_test, y_test) )

y_predicted_train_catDog = model10.predict_classes(X_train) 
y_predicted_test_catDog = model10.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/50
24500/24500 [==============================] - 11s 439us/step - loss: 8.0653 - acc: 0.4993 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 2/50
24500/24500 [==============================] - 10s 409us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 3/50
24500/24500 [==============================] - 10s 406us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 4/50
24500/24500 [==============================] - 10s 403us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 5/50
24500/24500 [==============================] - 10s 401us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 6/50
24500/24500 [==============================] - 10s 399us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 7/50
24500/24500 [==============================] - 10s 397us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.60

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
model10.summary()
model10.save('10initialized-model.keras')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 2304)              0         
__________

In [0]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.svm import SVC
from sklearn.decomposition import PCA
X = pd.read_csv("X.csv", sep = ' ', header = None , dtype = float)
X = X.values
y= pd.read_csv("y_bush_vs_others.csv" , header = None)
y_bush = y.values.ravel()
y= pd.read_csv("y_williams_vs_others.csv" , header = None)
y_williams = y.values.ravel()
print("X Shape",X.shape)
print("# of Bush's photos",np.sum(y_bush))
print("# of Bush's photos",np.sum(y_williams))

from sklearn.model_selection import train_test_split

X_reshaped=X.reshape((X.shape[0],64,64,1))
X_reshaped.shape

X Shape (13233, 4096)
# of Bush's photos 530
# of Bush's photos 52


(13233, 64, 64, 1)

In [0]:
from keras.models import load_model
modelForBush=load_model("7initialized-model.keras")
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_reshaped, 
             y_bush,shuffle = True ,test_size= 1./3, random_state=6099, stratify  = y_bush) # A20396099


modelForBush.fit(X_train_b,y_train_b, epochs=150, batch_size=32, validation_data = (X_test_b, y_test_b ) )

y_predicted_train_bush = modelForBush.predict_classes(X_train_b) 
y_predicted_test_bush = modelForBush.predict_classes(X_test_b) 
result_f1_train_bush_E70=f1_score(y_train_b, y_predicted_train_bush)
result_f1_test_bush_E70=f1_score(y_test_b, y_predicted_test_bush)
print("Bush F1 Score on Train : ",result_f1_train_bush_E70)
print("Bush F1 Score on Test : ",result_f1_test_bush_E70)

Train on 8822 samples, validate on 4411 samples
Epoch 1/150
8822/8822 [==============================] - 10s 1ms/step - loss: 0.3605 - acc: 0.9468 - val_loss: 0.6419 - val_acc: 0.9599
Epoch 2/150
8822/8822 [==============================] - 9s 1ms/step - loss: 0.1911 - acc: 0.9595 - val_loss: 0.2820 - val_acc: 0.9599
Epoch 3/150
8822/8822 [==============================] - 9s 1ms/step - loss: 0.1668 - acc: 0.9613 - val_loss: 0.2143 - val_acc: 0.9599
Epoch 4/150
8822/8822 [==============================] - 9s 1ms/step - loss: 0.1361 - acc: 0.9621 - val_loss: 0.6468 - val_acc: 0.9599
Epoch 5/150
8822/8822 [==============================] - 9s 1ms/step - loss: 0.1107 - acc: 0.9684 - val_loss: 0.2927 - val_acc: 0.8939
Epoch 6/150
8822/8822 [==============================] - 9s 1ms/step - loss: 0.1016 - acc: 0.9723 - val_loss: 0.1839 - val_acc: 0.9599
Epoch 7/150
8822/8822 [==============================] - 9s 1ms/step - loss: 0.0844 - acc: 0.9778 - val_loss: 0.0660 - val_acc: 0.9803
Epoch 

In [0]:
result_f1_train_bush_E70

In [0]:
result_f1_test_bush_E70

In [0]:
from keras.models import load_model
modelForWilliams=load_model("7initialized-model.keras")
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_reshaped, 
             y_bush,shuffle = True ,test_size= 1./3, random_state=6099, stratify  = y_williams) # A20396099


modelForWilliams.fit(X_train_w,y_train_w, epochs=150, batch_size=32, validation_data = (X_test_b, y_test_b ) )

y_predicted_train_will = modelForWilliams.predict_classes(X_train_w) 
y_predicted_test_will = modelForWilliams.predict_classes(X_test_w) 

result_f1_train_will_E100=f1_score(y_train_b, y_predicted_train_will)
result_f1_test_will_E100=f1_score(y_test_b, y_predicted_test_will)
print("Williams F1 Score on Train : ",result_f1_train_will_E100)
print("Williams F1 Score on Test : ",result_f1_test_will_E100)

Train on 8822 samples, validate on 4411 samples
Epoch 1/150
8822/8822 [==============================] - 11s 1ms/step - loss: 0.3606 - acc: 0.9455 - val_loss: 0.6468 - val_acc: 0.9599
Epoch 2/150
8822/8822 [==============================] - 9s 1ms/step - loss: 0.1729 - acc: 0.9583 - val_loss: 0.3379 - val_acc: 0.9599
Epoch 3/150
8352/8822 [===========================>..] - ETA: 0s - loss: 0.1354 - acc: 0.9614

In [0]:
result_f1_train_will_E100

In [0]:
result_f1_test_will_E100

In [22]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model11 = Sequential()

model11.add(Conv2D(64, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model11.add(MaxPooling2D(pool_size = (2, 2)))

model11.add(Conv2D(64, (3, 3),  activation = 'relu'))
model11.add(MaxPooling2D(pool_size = (2, 2)))




model11.add(Flatten())
model11.add(Dense(64, activation='relu'))
model11.add(Dropout(0.5))
model11.add(Dense(1, activation='sigmoid'))


model11.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model11.fit(X_train,y_train, epochs=20, batch_size=32, validation_data = (X_test, y_test) )




y_predicted_train_catDog = model11.predict_classes(X_train) 
y_predicted_test_catDog = model11.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/20
24500/24500 [==============================] - 14s 565us/step - loss: 7.9595 - acc: 0.5001 - val_loss: 8.4176 - val_acc: 0.4720
Epoch 2/20
 8896/24500 [=========>....................] - ETA: 8s - loss: 7.8763 - acc: 0.5060

KeyboardInterrupt: ignored

In [0]:
import cv2
import glob
import numpy as np

X_data = []
files = glob.glob ("train/*.jpg")
for myFile in files:
#     print(myFile)
    image = cv2.imread (myFile)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to greyscale
    X_data.append (gray_image)




KeyboardInterrupt: ignored

In [0]:
X = np.array(X_data)
print('X_data shape:', X.shape)
X.shape

X_data shape: (25000,)


25000

In [0]:
X_reshaped=X.reshape((X.shape[0],64,64,1))
X_reshaped.shape

In [0]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

from keras.layers import Conv2D, MaxPooling2D
import os
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


Using TensorFlow backend.


170500096/170498071 [==============================] - 115s 1us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:

import tensorflow as tf
x_train_reshaped = tf.image.resize_images(x_train, (64,64))
print(x_train_reshaped.shape)


x_test_reshaped = tf.image.resize_images(x_test, (64,64))
print(x_test_reshaped.shape)

x_train_reshaped= tf.image.rgb_to_grayscale(x_train_reshaped)
print(x_train_reshaped.shape)


x_test_reshaped= tf.image.rgb_to_grayscale(x_test_reshaped)
print(x_test_reshaped.shape)

y_train_ML = np.zeros((y_train.shape))
for cla in range(len(y_train)):
  if y_train[cla] == 1:
    y_train_ML[cla] = 1
  else:
    y_train_ML[cla] = 0
    
y_train_ML.shape


NameError: ignored

In [23]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model12 = Sequential()

model12.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model12.add(MaxPooling2D(pool_size = (2, 2)))

model12.add(Conv2D(32, (3, 3),  activation = 'relu'))
model12.add(MaxPooling2D(pool_size = (2, 2)))


model12.add(Conv2D(64, (3, 3),  activation = 'relu'))
model12.add(MaxPooling2D(pool_size = (2, 2)))


model12.add(Flatten())
model12.add(layers.Dropout(0.5))
model12.add(Dense(1, activation='sigmoid'))


model12.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model12.fit(X_train,y_train, epochs=100, batch_size=32, validation_data = (X_test, y_test) )

y_predicted_train_catDog = model12.predict_classes(X_train) 
y_predicted_test_catDog = model12.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/100
24500/24500 [==============================] - 11s 433us/step - loss: 1.1885 - acc: 0.5351 - val_loss: 0.6925 - val_acc: 0.5440
Epoch 2/100
24500/24500 [==============================] - 9s 381us/step - loss: 0.6876 - acc: 0.5708 - val_loss: 0.6472 - val_acc: 0.6500
Epoch 3/100
24500/24500 [==============================] - 9s 379us/step - loss: 0.6529 - acc: 0.6283 - val_loss: 0.5671 - val_acc: 0.7160
Epoch 4/100
24500/24500 [==============================] - 9s 380us/step - loss: 0.5938 - acc: 0.6874 - val_loss: 0.5643 - val_acc: 0.7380
Epoch 5/100
24500/24500 [==============================] - 9s 378us/step - loss: 0.5611 - acc: 0.7157 - val_loss: 0.4905 - val_acc: 0.7660
Epoch 6/100
24500/24500 [==============================] - 9s 383us/step - loss: 0.5342 - acc: 0.7397 - val_loss: 0.4934 - val_acc: 0.7680
Epoch 7/100
24500/24500 [==============================] - 9s 383us/step - loss: 0.5130 - acc: 0.7525 - val_loss: 0.4

In [26]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model13 = Sequential()

model13.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model13.add(MaxPooling2D(pool_size = (2, 2)))

model13.add(Conv2D(32, (3, 3),  activation = 'relu'))
model13.add(MaxPooling2D(pool_size = (2, 2)))


model13.add(Conv2D(64, (3, 3),  activation = 'relu'))
model13.add(MaxPooling2D(pool_size = (2, 2)))


model13.add(Flatten())
model13.add(Dense(64, activation='relu'))

model13.add(BatchNormalization())

model13.add(layers.Dropout(0.5))
model13.add(Dense(1, activation='sigmoid'))


model13.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
model13.fit(X_train,y_train, epochs=20, batch_size=32, validation_data = (X_test, y_test) )

y_predicted_train_catDog = model13.predict_classes(X_train) 
y_predicted_test_catDog = model13.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/20
24500/24500 [==============================] - 13s 547us/step - loss: 0.6272 - acc: 0.6587 - val_loss: 0.7157 - val_acc: 0.6320
Epoch 2/20
24500/24500 [==============================] - 12s 495us/step - loss: 0.4887 - acc: 0.7695 - val_loss: 0.4030 - val_acc: 0.8120
Epoch 3/20
24500/24500 [==============================] - 12s 495us/step - loss: 0.4367 - acc: 0.8013 - val_loss: 0.5950 - val_acc: 0.7180
Epoch 4/20
24500/24500 [==============================] - 12s 493us/step - loss: 0.3884 - acc: 0.8294 - val_loss: 0.6592 - val_acc: 0.7160
Epoch 5/20
24500/24500 [==============================] - 12s 493us/step - loss: 0.3567 - acc: 0.8449 - val_loss: 0.2971 - val_acc: 0.8860
Epoch 6/20
24500/24500 [==============================] - 12s 494us/step - loss: 0.3234 - acc: 0.8604 - val_loss: 0.3940 - val_acc: 0.8200
Epoch 7/20
24500/24500 [==============================] - 12s 493us/step - loss: 0.2995 - acc: 0.8731 - val_loss: 0.29

In [27]:
model13.summary()
model13.save('13initialized-model.keras')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 2304)              0         
__________

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from sklearn.metrics import f1_score
from keras import layers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dropout
from keras.layers import Dense,Flatten

# Model
model10 = Sequential()

model10.add(Conv2D(32, (3, 3), input_shape = (64, 64, 1), activation = 'relu'))
model10.add(MaxPooling2D(pool_size = (2, 2)))

model10.add(Conv2D(64, (3, 3),  activation = 'relu'))
model10.add(MaxPooling2D(pool_size = (2, 2)))


model10.add(Conv2D(64, (3, 3),  activation = 'relu'))
model10.add(MaxPooling2D(pool_size = (2, 2)))


model10.add(Flatten())
model10.add(Dense(1, activation='sigmoid'))


model10.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model10.fit(X_train,y_train, epochs=50, batch_size=32, validation_data = (X_test, y_test) )

y_predicted_train_catDog = model10.predict_classes(X_train) 
y_predicted_test_catDog = model10.predict_classes(X_test) 
result_f1_train=f1_score(y_train, y_predicted_train_catDog)
result_f1_test=f1_score(y_test, y_predicted_test_catDog)
print(" F1 Score on Train : ",result_f1_train)
print(" F1 Score on Test : ",result_f1_test)


Train on 24500 samples, validate on 500 samples
Epoch 1/50
24500/24500 [==============================] - 14s 579us/step - loss: 8.0619 - acc: 0.4995 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 2/50
24500/24500 [==============================] - 10s 398us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 3/50
24500/24500 [==============================] - 10s 399us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 4/50
24500/24500 [==============================] - 10s 398us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 5/50
24500/24500 [==============================] - 10s 403us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 6/50
24500/24500 [==============================] - 10s 403us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.6077 - val_acc: 0.5280
Epoch 7/50
24500/24500 [==============================] - 10s 400us/step - loss: 8.0683 - acc: 0.4994 - val_loss: 7.60

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
model10.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
__________

In [9]:
model10.layers

In [10]:
model10.inputs

[<tf.Tensor 'conv2d_4_input:0' shape=(?, 64, 64, 1) dtype=float32>]

In [11]:
model10.outputs

[<tf.Tensor 'dense_2/Sigmoid:0' shape=(?, 1) dtype=float32>]

In [0]:
model10.save('newmodel.keras')